In [1]:
%load_ext autoreload
%autoreload 2

In [50]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp

from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from preprocessing import process_dataset, expand_cabin, expand_passenger_id_to_group, convert_to_categorical, get_where_all_are_non_null

sns.set_theme()

tfd = tfp.distributions
tfb = tfp.bijectors

DATASET_PATH = "/Users/dincaus/datasets/spaceship-titanic"

TRAIN_PATH = os.path.join(DATASET_PATH, "train.csv")
TEST_PATH = os.path.join(DATASET_PATH, "test.csv")

In [29]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

all_data = pd.concat([train_df, test_df])

In [35]:
all_data[["Deck", "CabinNumber", "Side"]] = expand_cabin(all_data)
all_data["PassengerGroup"] = expand_passenger_id_to_group(all_data)

all_data = all_data.drop(columns=["PassengerId", "Transported", "Name", "Cabin"])

In [36]:
CATEGORICAL_FEATURES = ["HomePlanet", "CrypSleep", "Destination", "VIP", "Deck", "Side", "PassengerGroup"]
NUMERICAL_FEATURES = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "CabinNumber"]

In [48]:
knn_impute = KNNImputer(n_neighbors=2)
knn_impute.fit(all_data[NUMERICAL_FEATURES].to_numpy())
knn_impute.transform(all_data[NUMERICAL_FEATURES].to_numpy())

array([[  39. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
       [  24. ,  109. ,    9. , ...,  549. ,   44. ,    0. ],
       [  58. ,   43. , 3576. , ..., 6715. ,   49. ,    0. ],
       ...,
       [  37. ,    0. ,    0. , ...,    0. ,    0. ,  296. ],
       [  35.5,    0. , 2680. , ...,    0. ,  523. ,  297. ],
       [  43. ,    0. ,    0. , ...,    0. ,    0. , 1498. ]])